# Pydeck Earth Engine Introduction

This is an introduction to using [Pydeck](https://pydeck.gl) and [Deck.gl](https://deck.gl) with [Google Earth Engine](https://earthengine.google.com/) in Jupyter Notebooks.

If you wish to run this locally, you'll need to install some dependencies. Installing into a new Conda environment is recommended. To create and enter the environment, run:
```
conda create -n pydeck-ee -c conda-forge python jupyter notebook pydeck earthengine-api requests -y
source activate pydeck-ee
jupyter nbextension install --sys-prefix --symlink --overwrite --py pydeck
jupyter nbextension enable --sys-prefix --py pydeck
```
then open Jupyter Notebook with `jupyter notebook`.

Now in a Python Jupyter Notebook, let's first import required packages:

In [1]:
from pydeck_earthengine_layers import EarthEngineLayer
import pydeck as pdk
import requests
import ee

## Authentication

Using Earth Engine requires authentication. If you don't have a Google account approved for use with Earth Engine, you'll need to request access. For more information and to sign up, go to https://signup.earthengine.google.com/.

If you haven't used Earth Engine in Python before, you'll need to run the following authentication command. If you've previously authenticated in Python or the command line, you can skip the next line.

Note that this creates a prompt which waits for user input. If you don't see a prompt, you may need to authenticate on the command line with `earthengine authenticate` and then return here, skipping the Python authentication.

In [ ]:
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

## Create Map

Next it's time to create a map. Here we create an `ee.Image` object

In [ ]:
# Initialize objects
ee_layers = []
view_state = pdk.ViewState(latitude=37.7749295, longitude=-122.4194155, zoom=10, bearing=0, pitch=45)

In [4]:
# %%
# Add Earth Engine dataset
# Load three NAIP quarter quads in the same location, different times.
naip2004_2012 = ee.ImageCollection('USDA/NAIP/DOQQ') \
  .filterBounds(ee.Geometry.Point(-71.08841, 42.39823)) \
  .filterDate('2004-07-01', '2012-12-31') \
  .select(['R', 'G', 'B'])

# Temporally composite the images with a maximum value function.
composite = naip2004_2012.max()
view_state = pdk.ViewState(longitude=-71.12532, latitude=42.3712, zoom=12)
ee_layers.append(EarthEngineLayer(ee_object=composite, vis_params={}))


# Load four 2012 NAIP quarter quads, different locations.
naip2012 = ee.ImageCollection('USDA/NAIP/DOQQ') \
  .filterBounds(ee.Geometry.Rectangle(-71.17965, 42.35125, -71.08824, 42.40584)) \
  .filterDate('2012-01-01', '2012-12-31')

# Spatially mosaic the images in the collection and display.
mosaic = naip2012.mosaic()
view_state = pdk.ViewState(longitude=-71.12532, latitude=42.3712, zoom=12)
ee_layers.append(EarthEngineLayer(ee_object=mosaic, vis_params={}))


# Load a NAIP quarter quad, display.
naip = ee.Image('USDA/NAIP/DOQQ/m_4207148_nw_19_1_20120710')
view_state = pdk.ViewState(longitude=-71.0915, latitude=42.3443, zoom=14)
ee_layers.append(EarthEngineLayer(ee_object=naip, vis_params={}))

# Create the NDVI and NDWI spectral indices.
ndvi = naip.normalizedDifference(['N', 'R'])
ndwi = naip.normalizedDifference(['G', 'N'])

# Create some binary images from thresholds on the indices.
# This threshold is designed to detect bare land.
bare1 = ndvi.lt(0.2).And(ndwi.lt(0.3))
# This detects bare land with lower sensitivity. It also detects shadows.
bare2 = ndvi.lt(0.2).And(ndwi.lt(0.8))

# Define visualization parameters for the spectral indices.
ndviViz = {'min': -1, 'max': 1, 'palette': ['FF0000', '00FF00']}
ndwiViz = {'min': 0.5, 'max': 1, 'palette': ['00FFFF', '0000FF']}

# Mask and mosaic visualization images.  The last layer is on top.
mosaic = ee.ImageCollection([
  # NDWI > 0.5 is water.  Visualize it with a blue palette.
  ndwi.updateMask(ndwi.gte(0.5)).visualize(**ndwiViz),
  # NDVI > 0.2 is vegetation.  Visualize it with a green palette.
  ndvi.updateMask(ndvi.gte(0.2)).visualize(**ndviViz),
  # Visualize bare areas with shadow (bare2 but not bare1) as gray.
  bare2.updateMask(bare2.And(bare1.Not())).visualize(**{'palette': ['AAAAAA']}),
  # Visualize the other bare areas as white.
  bare1.updateMask(bare1).visualize(**{'palette': ['FFFFFF']}),
]).mosaic()
ee_layers.append(EarthEngineLayer(ee_object=mosaic, vis_params={}))



# # This function masks clouds in Landsat 8 imagery.
# maskClouds = function(image) {
#   scored = ee.Algorithms.Landsat.simpleCloudScore(image)
#   return image.updateMask(scored.select(['cloud']).lt(20))
# }

# # This function masks clouds and adds quality bands to Landsat 8 images.
# addQualityBands = function(image) {
#   return maskClouds(image)
#     # NDVI \
#     .addBands(image.normalizedDifference(['B5', 'B4']))
#     # time in days \
#     .addBands(image.metadata('system:time_start'))
# }

# # Load a 2014 Landsat 8 ImageCollection.
# # Map the cloud masking and quality band function over the collection.
# collection = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA') \
#   .filterDate('2014-06-01', '2014-12-31') \
#   .map(addQualityBands)

# # Create a cloud-free, most recent value composite.
# recentValueComposite = collection.qualityMosaic('system:time_start')

# # Create a greenest pixel composite.
# greenestPixelComposite = collection.qualityMosaic('nd')

# # Display the results.
# Map.setCenter(-122.374, 37.8239, 12) # San Francisco Bay
# vizParams = {'bands': ['B5', 'B4', 'B3'], 'min': 0, 'max': 0.4}
# Map.addLayer(recentValueComposite, vizParams, 'recent value composite')
# Map.addLayer(greenestPixelComposite, vizParams, 'greenest pixel composite')

# # Compare to a cloudy image in the collection.
# cloudy = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140825')
# Map.addLayer(cloudy, vizParams, 'cloudy')





Then just pass these layers to a `pydeck.Deck` instance, and call `.show()` to create a map:

In [8]:
r = pdk.Deck(layers=ee_layers, initial_view_state=view_state)
r.show()

DeckGLWidget(custom_libraries=[{'libraryName': 'EarthEngineLayerLibrary', 'resourceUri': 'https://cdn.jsdelivr…